<div style="background-color: #333; padding: 40px; border: 2px solid #ffd700; border-radius: 10px; color: #ffd700; text-align: center; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">

<h1 style="font-size: 48px; font-weight: bold; color: #ffd700;">LLM Classification finetuning DeBERTA</h1>

<img src="https://cdn.arstechnica.net/wp-content/uploads/2023/12/GettyImages-152404829-scaled.jpg" alt="Chatbot arena" style="width: 500px; margin: 20px auto; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">
    
</div>

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">Table of content</div>

<ul class="list-group" id="list-tab" role="tablist">
    <li><a href="#0.-Introduction">0. Introduction</a></li><br>
    <li><a href="#1.-Import-Libraries">1. Import Libraries</a></li><br>
    <li><a href="#2.-Data-Loading-&-Inspection">2. Data Loading & Inspection</a></li><br>
    <li><a href="#3.-Text-Preprocessing">3. Text Preprocessing</a></li><br>
    <li><a href="#4.-Dataset-Preparation">4. Dataset Preparation</a></li><br>
    <li><a href="#5.-Model-Building">5. Model Building</a></li><br>
    <li><a href="#6.-Training-&-Evaluation">6. Training & Evaluation</a></li><br>
    <li><a href="#7.-Prediction-&-Submission">7. Prediction & Submission</a></li><br>
</ul>

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">0. Introduction</div>

### Intro : 

In the rapidly evolving world of large language models (LLMs), one of the most critical challenges is ensuring that AI-generated responses align with human preferences. While modern chatbots can produce fluent and coherent text, not all responses are equally engaging, helpful, or satisfying to users. This Kaggle competition tackles this challenge head-on by leveraging real-world data from Chatbot Arena, where users compare responses from different LLMs and choose their preferred one.

### Competition Overview :

The goal is to predict which LLM response a human judge will prefer in a head-to-head battle. Each conversation consists of:

A user prompt (the input given to the chatbots).

Two LLM-generated responses (anonymous models competing against each other).

A human preference label (indicating which response was preferred).

This task mirrors Reinforcement Learning from Human Feedback (RLHF), a key technique for aligning AI with human values. Successfully predicting preferences helps improve reward models, which are essential for training better chatbots.

### Key challenges :

1. Biases in Human Judgments

    * Position bias: Users may favor the first or second response due to ordering.

    * Verbosity bias: Longer responses might be preferred even if less accurate.

    * Self-enhancement bias: Models may subtly promote themselves.

2. Model Generalization

    * The test set (~25K samples) requires robust predictions beyond the training data (55K samples).

3. Interpretable Preference Modeling

    * Understanding why users prefer certain responses can guide better LLM fine-tuning.
  
### Why this matters :

Improving preference prediction models directly enhances:

* Chatbot training.

* User satisfaction (by aligning AI with human expectations).

* Fairness & robustness (reducing biases in AI judgments).

By competing in this challenge, we contribute to the future of human-aligned AI assistants—making them not just smarter, but also more attuned to what users truly want.

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">1. Import Libraries</div>

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

# Configure GPU memory growth (prevents OOM errors)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_nlp
from sklearn.model_selection import train_test_split
from keras import backend as K

# Configuration
MODEL_NAME = "deberta_v3_extra_small_en"
SEQUENCE_LENGTH = 128
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 5e-6

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">2. Data Loading & Inspection</div>

In [ ]:
# Load datasets
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# Quick inspection
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()

In [ ]:
# Basic validation checks
def check_data(df, name):
    print(f"\n{name} Data Summary:")
    print("- Missing values:", df.isna().sum().sum())
    print("- Duplicates:", df.duplicated().sum())
    print("- Target distribution:")
    print(df[['winner_model_a', 'winner_model_b', 'winner_tie']].mean())


check_data(train_df, "Train")

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">3. Text Preprocessing</div>

In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(MODEL_NAME)
        
    def clean_text(self, text):
        """Normalize text for DeBERTa"""
        text = str(text)
        text = re.sub(r"\s+", " ", text)  # Collapse whitespace
        text = re.sub(r"[^\x00-\x7F]+", " ", text)  # Remove non-ASCII
        return text.strip()
    
    def create_input_pairs(self, row):
        """Format prompt-response pairs"""
        clean_prompt = self.clean_text(row['prompt'])
        return [
            f"Prompt: {clean_prompt} {self.tokenizer.sep_token} Response: {self.clean_text(row['response_a'])}",
            f"Prompt: {clean_prompt} {self.tokenizer.sep_token} Response: {self.clean_text(row['response_b'])}"
        ]

# Initialize processor
processor = TextPreprocessor()

In [ ]:
# Apply preprocessing
train_df['inputs'] = train_df.apply(processor.create_input_pairs, axis=1)
test_df['inputs'] = test_df.apply(processor.create_input_pairs, axis=1)

# Create labels (0: model_a wins, 1: model_b wins, 2: tie)
train_df['label'] = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)
train_df['label'] = train_df['label'].map({'winner_model_a':0, 'winner_model_b':1, 'winner_tie':2})

# Preview processed data
train_df[['inputs', 'label']].head()

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">4. Dataset Preparation</div>

In [ ]:
# Train/validation split
train_df, valid_df = train_test_split(
    train_df, 
    test_size=0.1, 
    stratify=train_df['label'],
    random_state=42
)

# Create TensorFlow datasets with proper input pair handling
def create_dataset(text_pairs, labels=None, preprocessor=None):
    """Convert to optimized TF Dataset with proper input pair handling"""
    AUTO = tf.data.AUTOTUNE
    
    # Convert to TensorFlow Dataset
    if labels is not None:
        ds = tf.data.Dataset.from_tensor_slices((text_pairs, labels))
        ds = ds.shuffle(1000)
    else:
        ds = tf.data.Dataset.from_tensor_slices(text_pairs)
    
    # Preprocessing function
    def preprocess_pair(text_pair, label=None):
        """Convert raw text pairs to model-ready format"""
        # Tokenize each response separately
        processed_a = preprocessor(text_pair[0])  # {'token_ids': ..., 'padding_mask': ...}
        processed_b = preprocessor(text_pair[1])
        
        # Stack to create (2, seq_len) tensors
        model_inputs = {
            "token_ids": tf.stack([processed_a["token_ids"], processed_b["token_ids"]], axis=0),
            "padding_mask": tf.stack([processed_a["padding_mask"], processed_b["padding_mask"]], axis=0)
        }
        return (model_inputs, label) if label is not None else model_inputs
    
    # Apply preprocessing and batching
    ds = ds.map(preprocess_pair, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTO)
    return ds

# Initialize preprocessor
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    MODEL_NAME,
    sequence_length=SEQUENCE_LENGTH
)

# Prepare all datasets
train_ds = create_dataset(
    train_df['inputs'].tolist(), 
    tf.keras.utils.to_categorical(train_df['label']),
    preprocessor=preprocessor
)
valid_ds = create_dataset(
    valid_df['inputs'].tolist(), 
    tf.keras.utils.to_categorical(valid_df['label']),
    preprocessor=preprocessor
)
test_ds = create_dataset(
    test_df['inputs'].tolist(),
    preprocessor=preprocessor
)

In [ ]:
# Check a batch from your dataset
for batch in train_ds.take(1):
    inputs, labels = batch
    print("Token IDs shape:", inputs["token_ids"].shape)  # Should be (batch_size, 2, 128)
    print("Mask shape:", inputs["padding_mask"].shape)
    print("Example token_ids[0,0,:5]:", inputs["token_ids"][0,0,:5])  # First 5 tokens of response_a

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">5. Model Building</div>

In [ ]:
def build_deberta_classifier():
    with tf.device('/GPU:0'):
        # Define input layers
        token_ids = tf.keras.layers.Input(
            shape=(2, SEQUENCE_LENGTH), 
            dtype=tf.int32,
            name="token_ids"
        )
        padding_mask = tf.keras.layers.Input(
            shape=(2, SEQUENCE_LENGTH),
            dtype=tf.int32,
            name="padding_mask"
        )
        
        inputs = {"token_ids": token_ids, "padding_mask": padding_mask}
        
        # Initialize backbone
        backbone = keras_nlp.models.DebertaV3Backbone.from_preset(MODEL_NAME)
        
        # Process both responses
        def process_response(inputs, index):
            return {
                "token_ids": inputs["token_ids"][:, index, :],
                "padding_mask": inputs["padding_mask"][:, index, :]
            }
        
        emb_a = backbone(process_response(inputs, 0))
        emb_b = backbone(process_response(inputs, 1))
        
        # Classification head
        combined = tf.keras.layers.Concatenate()([emb_a, emb_b])
        x = tf.keras.layers.GlobalAveragePooling1D()(combined)
        x = tf.keras.layers.Dropout(0.5)(x)
        outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# 2. Build and compile the model
model = build_deberta_classifier()

model.compile(
    optimizer=tf.keras.optimizers.AdamW(LEARNING_RATE, weight_decay=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy", "categorical_crossentropy"]
)

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">6. Training & Evaluation</div>

In [ ]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.weights.h5", save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
]

# Train model
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Plot training history
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.title("Training History")
plt.show()

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">7. Prediction & Submission</div>

In [ ]:
# Generate predictions
test_preds = model.predict(test_ds)
test_df['prediction'] = np.argmax(test_preds, axis=1)

# Create submission
submission = pd.DataFrame({
            "id": test_df.id,
            "winner_model_a": test_preds[:, 0],
            "winner_model_b": test_preds[:, 1],
            "winner_tie": test_preds[:, 2]
        })
submission.to_csv("submission.csv", index=False)
print("Submission saved!")
submission

<div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #001f3f; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">If you found this work helpful or valuable, I would greatly appreciate an upvote.</div>